In [10]:
import pandas as pd
import re
from typing import Optional, Tuple

# ============= 配置 =============
SOURCE_ETF = 'WLS'  # 固定值
INPUT_FILE = 'holdings.xlsx'  # 你的文件名
OUTPUT_FILE = 'holdings_cleaned.csv'

# ============= 辅助函数 =============

def is_section_row(row: pd.Series) -> bool:
    """判断是否为section行：Name列包含'members'且Ticker为空"""
    ticker_empty = pd.isna(row.get('Ticker')) or str(row.get('Ticker', '')).strip() == ''
    name = str(row.get('Name', ''))
    return ticker_empty and 'members' in name.lower()

def extract_section_name(text: str) -> str:
    """提取section名称，去掉括号内容
    Example: 'Information Technology (1362 members)' -> 'Information Technology'
    """
    match = re.search(r'^(.+?)\s*\(', text)
    if match:
        return match.group(1).strip()
    return text.strip()

def extract_ticker_and_country(ticker_text: str) -> Tuple[Optional[str], Optional[str]]:
    """从 'NVDA UW Equity' 提取 ('NVDA', 'UW')"""
    if pd.isna(ticker_text):
        return None, None
    
    parts = str(ticker_text).strip().split()
    if len(parts) >= 2:
        return parts[0], parts[1]  # ticker, country_code
    elif len(parts) == 1:
        return parts[0], None
    return None, None

def clean_numeric(value) -> Optional[float]:
    """清理数字：移除逗号，转换为float"""
    if pd.isna(value):
        return None
    try:
        # 移除逗号并转换
        cleaned = str(value).replace(',', '')
        return float(cleaned)
    except:
        return None

# ============= 主处理逻辑 =============

def process_excel_to_csv(input_file: str, output_file: str, source_etf: str):
    """主函数：处理Excel并输出CSV"""
    
    # 1. 读取数据
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    print(f"原始数据: {len(df)} 行")
    
    # 2. 准备结果容器
    results = []
    current_section = None
    
    # 3. 逐行处理（状态机）
    for idx, row in df.iterrows():
        # 3.1 识别section行
        if is_section_row(row):
            current_section = extract_section_name(row['Name'])
            print(f"发现section: {current_section}")
            continue
        
        # 3.2 检查是否为有效股票行
        ticker, country_code = extract_ticker_and_country(row.get('Ticker'))
        
        # 跳过无效行
        if not ticker or (pd.isna(row.get('Name')) and pd.isna(row.get('Weight'))):
            continue
        
        # 3.3 构建结果记录
        results.append({
            'ticker': ticker,
            'source_etf': source_etf,
            'country_code': country_code,
            'region': '',  # 暂时留空
            'section': current_section or '',
            'weight': clean_numeric(row.get('Weight')),
            'shares': clean_numeric(row.get('Shares')),
            'price': clean_numeric(row.get('Price'))
        })
    
    # 4. 转换为DataFrame并保存
    result_df = pd.DataFrame(results)
    print(f"\n处理完成: {len(result_df)} 条有效记录")
    print(f"\n预览前5条:")
    print(result_df.head())
    
    # 5. 输出CSV
    result_df.to_csv(output_file, index=False)
    print(f"\n已保存到: {output_file}")
    
    return result_df

# ============= 执行 =============

if __name__ == '__main__':
    df_result = process_excel_to_csv(INPUT_FILE, OUTPUT_FILE, SOURCE_ETF)

读取文件: holdings.xlsx
原始数据: 10427 行
发现section: Information Technology
发现section: Financials
发现section: Industrials
发现section: Consumer Discretionary
发现section: Health Care
发现section: Communication Services
发现section: Consumer Staples
发现section: Materials
发现section: Energy
发现section: Utilities
发现section: Real Estate
发现section: N.A.

处理完成: 10414 条有效记录

预览前5条:
  ticker source_etf country_code region                 section    weight  \
0   NVDA        WLS           UW         Information Technology  4.330869   
1   MSFT        WLS           UW         Information Technology  3.723246   
2   AAPL        WLS           UW         Information Technology  3.562665   
3   AVGO        WLS           UW         Information Technology  1.526520   
4   2330        WLS           TT         Information Technology  0.837250   

      shares    price  
0  23444.971   192.57  
1   7429.881   522.40  
2  14619.625   254.04  
3   4612.342   345.02  
4  18536.260  1440.00  

已保存到: holdings_cleaned.csv


In [12]:
# 修正股数单位并重新计算市值
import pandas as pd

# 读取数据
df = pd.read_csv('holdings_cleaned.csv')

print("=== 修正股数单位 ===")
print("假设股数单位是千股，需要乘以1000")
print("例如：NVDA 23444.971 千股 = 23,444,971 股")

# 修正股数（乘以1000，从千股转为股）
df['shares_corrected'] = df['shares'] * 1000

# 重新计算市值
df['market_cap_corrected'] = df['shares_corrected'] * df['price']

print(f"\n=== 修正后的市值统计 ===")
print(df['market_cap_corrected'].describe())

print(f"\n=== 修正后的阈值分析 ===")
threshold_10b = 1e9  # 10亿
threshold_1b = 1e9    # 10亿

print(f"市值 > {threshold_10b/1e9:.0f}B 的股票数量: {len(df[df['market_cap_corrected'] > threshold_10b])}")
print(f"市值 > {threshold_1b/1e9:.0f}B 的股票数量: {len(df[df['market_cap_corrected'] > threshold_1b])}")

print(f"\n=== 修正后的最大市值股票 ===")
top_10 = df.nlargest(10, 'market_cap_corrected')[['ticker', 'shares_corrected', 'price', 'market_cap_corrected']]
print(top_10)

# 使用修正后的市值进行过滤
df_largecap = df[df['market_cap_corrected'] > threshold_10b].copy()
print(f"\n=== 最终结果 ===")
print(f"市值大于10B的股票数量: {len(df_largecap)}")

if len(df_largecap) > 0:
    print("\n前5只大市值股票:")
    print(df_largecap[['ticker', 'shares_corrected', 'price', 'market_cap_corrected']].head())
    
    # 保存结果
    df_largecap.to_csv('holdings_largecap_gt1bn.csv', index=False)
    print(f"\n已保存到 holdings_largecap_gt1bn.csv")
else:
    print("仍然没有找到市值大于10B的股票，可能需要进一步调整阈值或检查数据")


=== 修正股数单位 ===
假设股数单位是千股，需要乘以1000
例如：NVDA 23444.971 千股 = 23,444,971 股

=== 修正后的市值统计 ===
count    1.041400e+04
mean     6.981449e+08
std      8.243694e+09
min      4.939200e+04
25%      3.814589e+06
50%      1.286967e+07
75%      9.274314e+07
max      4.593896e+11
Name: market_cap_corrected, dtype: float64

=== 修正后的阈值分析 ===
市值 > 1B 的股票数量: 670
市值 > 1B 的股票数量: 670

=== 修正后的最大市值股票 ===
      ticker  shares_corrected     price  market_cap_corrected
9     005930         4866415.0   94400.0          4.593896e+11
1516    BBCA        52346754.0    7400.0          3.873660e+11
1571    BBRI        70121016.0    3730.0          2.615514e+11
28    000660          487029.0  428000.0          2.084484e+11
8098    TPIA        22219727.0    8100.0          1.799798e+11
9124    DSSA         1573762.0  106100.0          1.669761e+11
1668    BMRI        36538840.0    4250.0          1.552901e+11
9484    BREN        15549954.0    9850.0          1.531670e+11
6966    TLKM        47373012.0    3040.0          

In [ ]:
vgkTickers = [
  #  Netherlands
  'ASML', 'ASML.AS',
  #  Germany  
  'SAP', 'SAP.DE', 'SIE.DE', 'ALV.DE', 'DTE.DE', 'MBG.DE', 'BMW.DE', 'BAS.DE', 
  'VOW3.DE', 'ADS.DE', 'MUV2.DE', 'DB1.DE', 'CBK.DE', 'HEN3.DE', 'PAH3.DE',
  #  UK
  'AZN', 'AZN.L', 'SHEL.L', 'SHEL', 'HSBA.L', 'ULVR.L', 'DGE.L', 'BP.L', 
  'GSK.L', 'RIO.L', 'LSEG.L', 'NG.L', 'BARC.L', 'LLOY.L', 'VOD.L', 'PRU.L',
  'BT-A.L', 'REL.L', 'AV.L', 'RKT.L', 'IMB.L', 'BA.L', 'GLEN.L', 'AAL.L',
  'EXPN.L', 'CRDA.L', 'BDEV.L', 'RR.L', 'STAN.L', 'LGEN.L', 'INF.L', 'SSE.L',
  #  Switzerland
  'NESN.SW', 'NOVN.SW', 'ROG.SW', 'UHR.SW', 'UBSG.SW', 'ABBN.SW', 'ZURN.SW',
  'CSGN.SW', 'GIVN.SW', 'SREN.SW', 'SCMN.SW', 'LONN.SW', 'SLHN.SW', 'GEBN.SW',
  #  France
  'MC.PA', 'OR.PA', 'TTE.PA', 'SAN.PA', 'AIR.PA', 'BNP.PA', 'SAF.PA', 'SU.PA',
  'EL.PA', 'CS.PA', 'CAP.PA', 'DG.PA', 'BN.PA', 'VIV.PA', 'RI.PA', 'EN.PA',
  'SGO.PA', 'GLE.PA', 'ACA.PA', 'DSY.PA', 'KER.PA', 'RMS.PA', 'STM.PA', 'AI.PA',
  #  Denmark
  'NOVO-B.CO', 'ORSTED.CO', 'DSV.CO', 'MAERSK-B.CO', 'COLB.CO', 'DEMANT.CO',
  'GMAB.CO', 'VWS.CO', 'TRYG.CO', 'JYSK.CO',
  #  Spain
  'ITX.MC', 'IBE.MC', 'SAN.MC', 'BBVA.MC', 'TEF.MC', 'REP.MC', 'AMS.MC', 'ENG.MC',
  'FER.MC', 'IAG.MC', 'CABK.MC', 'SAB.MC', 'GRF.MC',
  #  Italy
  'ENI.MI', 'ISP.MI', 'ENEL.MI', 'UCG.MI', 'G.MI', 'TIT.MI', 'STM.MI', 'RACE.MI',
  'MB.MI', 'LDO.MI', 'BAMI.MI', 'TEN.MI', 'PRY.MI', 'CNHI.MI',
  #  Sweden
  'VOLV-B.ST', 'ERIC-B.ST', 'AZN.ST', 'ABB.ST', 'ATCO-A.ST', 'SEB-A.ST', 'HM-B.ST',
  'SAND.ST', 'SWED-A.ST', 'ASSA-B.ST', 'INVE-B.ST', 'HEXA-B.ST', 'ESSITY-B.ST',
  #  Norway
  'EQNR.OL', 'DNB.OL', 'MOWI.OL', 'TEL.OL', 'ORK.OL', 'YAR.OL', 'SALM.OL',
  #  Finland
  'NOKIA.HE', 'NESTE.HE', 'FORTUM.HE', 'STERV.HE', 'UPM.HE', 'SAMPO.HE',
  #  Belgium
  'ABI.BR', 'KBC.BR', 'UCB.BR', 'ACKB.BR', 'SOLB.BR',
  #  Ireland
  'CRH.I', 'SMDS.I', 'FLT.I', 'RYA.I', 'GLEN.I', 'CPL.I',
  #  Portugal
  'EDP.LS', 'GALP.LS', 'BCP.LS', 'JERONIMO.LS',
  #  Austria
  'OMV.VI', 'VOE.VI', 'ANDR.VI', 'POST.VI',
  #  Poland
  'PKO.WA', 'PZU.WA', 'PEO.WA', 'PKN.WA', 'KGH.WA',
  #  Greece
  'OPAP.AT', 'ALPHA.AT', 'ETE.AT', 'PPC.AT'
]



In [3]:
import pandas as pd

SOURCE_VGK = 'VGK'
REGION_VGK = 'developed'

# Export vgkTickers to CSV with required columns
output_path = '/Users/wenjiaqi/Downloads/bloomberg-competition/data/universes/vgk_tickers.csv'
df = pd.DataFrame({
    'ticker': vgkTickers,
    'source_etf': [SOURCE_VGK] * len(vgkTickers),
    'region': [REGION_VGK] * len(vgkTickers),
})
df.to_csv(output_path, index=False)
print(f"Saved {len(df)} rows to {output_path}")


Saved 178 rows to /Users/wenjiaqi/Downloads/bloomberg-competition/data/universes/vgk_tickers.csv
